In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras as keras

Read datasets (train & test) from https://grouplens.org/datasets/movielens/100k/ 

In [2]:
train_data =  pd.read_csv('./content/u1.base', sep = '\t', names = ['userId','movieId','rating','timestamp'])
train_data.head()

,userId,movieId,rating,timestamp
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712


In [3]:
test_data =  pd.read_csv('./content/u1.test', sep = '\t', names = ['userId','movieId','rating','timestamp'])
test_data.head()

,userId,movieId,rating,timestamp
0,1,6,5,887431973
1,1,10,3,875693118
2,1,12,5,878542960
3,1,14,5,874965706
4,1,17,3,875073198


Let's know our data better:


In [4]:
train_data.describe()

,userId,movieId,rating,timestamp
count,80000.00000,80000.000000,80000.000000,8.000000e+04
mean,525.16570,425.705200,3.528350,8.835645e+08
std,255.94956,331.383936,1.118565,5.318611e+06
min,1.00000,1.000000,1.000000,8.747247e+08
25%,334.00000,175.000000,3.000000,8.794556e+08
50%,535.00000,321.000000,4.000000,8.828441e+08
75%,744.00000,631.000000,4.000000,8.882676e+08
max,943.00000,1682.000000,5.000000,8.932866e+08


In [5]:
test_data.describe()

,userId,movieId,rating,timestamp
count,20000.00000,20000.000000,20000.00000,2.000000e+04
mean,211.76095,424.829850,3.53590,8.833864e+08
std,121.64690,328.452907,1.15368,5.441471e+06
min,1.00000,1.000000,1.00000,8.747247e+08
25%,99.00000,174.000000,3.00000,8.793730e+08
50%,222.00000,322.000000,4.00000,8.824747e+08
75%,308.00000,633.000000,4.00000,8.882045e+08
max,462.00000,1591.000000,5.00000,8.932777e+08


In [6]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80000 entries, 0 to 79999
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   userId     80000 non-null  int64
 1   movieId    80000 non-null  int64
 2   rating     80000 non-null  int64
 3   timestamp  80000 non-null  int64
dtypes: int64(4)
memory usage: 2.4 MB


In [7]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   userId     20000 non-null  int64
 1   movieId    20000 non-null  int64
 2   rating     20000 non-null  int64
 3   timestamp  20000 non-null  int64
dtypes: int64(4)
memory usage: 625.1 KB


In [8]:
print("no of ratings:", train_data.shape[0])
print("no of unique users:",train_data.userId.unique().shape[0])
print("no of unique movies:", train_data.movieId.unique().shape[0])
print("maximum rating:", train_data.rating.max())
print("minimum rating:", train_data.rating.min())

no of ratings: 80000
no of unique users: 943
no of unique movies: 1650
maximum rating: 5
minimum rating: 1


In [9]:
print("no of ratings:", test_data.shape[0])
print("no of unique users:",test_data.userId.unique().shape[0])
print("no of unique movies:", test_data.movieId.unique().shape[0])
print("maximum rating:", test_data.rating.max())
print("minimum rating:", test_data.rating.min())

no of ratings: 20000
no of unique users: 459
no of unique movies: 1410
maximum rating: 5
minimum rating: 1


 Using matrix factorization, we can find some latent features that can determine how a user rates a movie. We decompose the matrix into constituent parts in such a way that the product of these parts generates the original matrix.

Obs:

    np.random.normal = random samples from a normal Gaussian distribution. (scale = standard deviation)
    np.newaxis =  is used to increase the dimension of the existing array by one more dimension


In [10]:
def build_sparse_tensor(np_array, indices, shape):
  return tf.SparseTensor(
                indices=indices,
                values=np_array,
                dense_shape=[shape[0], shape[1]])

In [11]:
class MF_tensorflow():
  # Initializing the user-movie rating matrix, no. of latent features, alpha - learning rate and beta - regularization parameter for bias.
    def __init__(self, R, K, alpha, iterations, beta):
        self.R = R
        self.num_users, self.num_items = R.dense_shape #tupla
        self.K = K
        self.alpha = alpha
        self.iterations = iterations
        self.optimizer = tf.optimizers.Adam(alpha)
        self.beta = beta

    # Initializing user-feature and movie-feature matrix 
    def train(self):
        self.P = tf.Variable(tf.random.normal(shape = [self.num_users, self.K]), dtype=tf.float32, trainable=True)
        self.Q = tf.Variable(tf.random.normal(shape = [self.K, self.num_items]), dtype=tf.float32, trainable=True)

        nr_validation = 10

        # Stochastic gradient descent for given number of iterations
        training_process = []
        propr = (self.K, self.alpha, self.beta)

        loss_hist = np.zeros(nr_validation, np.float32)

        for i in range(self.iterations):
          predictions_matrix = tf.gather_nd(self.full_matrix(),self.R.indices)
          predictions_matrix = build_sparse_tensor(tf.gather_nd(tf.matmul(self.P, self.Q),self.R.indices), self.R.indices, self.R.dense_shape)
          # print(predictions_matrix)

          # loss = lambda: tf.losses.mean_squared_error(tf.sparse.to_dense(self.R), tf.sparse.to_dense(predictions_matrix))
          
          loss = lambda: tf.reduce_sum(tf.losses.mean_squared_error(self.R.values, tf.gather_nd(tf.matmul(self.P, self.Q),self.R.indices))) + self.regularizationL2()
          # loss = lambda: tf.reduce_sum(tf.pow( tf.math.subtract(tf.sparse.to_dense(self.R),tf.sparse.to_dense(build_sparse_tensor(tf.gather_nd(tf.matmul(self.P, self.Q),self.R.indices), self.R.indices, self.R.dense_shape))),2)/nonzero)
          # print(32*"-")
          self.optimizer.minimize(loss, var_list = [self.P, self.Q])
          # print(32*"-")

          nrmse = loss()
          # training_process.append((i, nrmse))
          training_process = (propr,i, nrmse)
          # print(i)
          loss_hist[i%nr_validation] = nrmse
          if i % nr_validation == 0 and i != 0:
            # print("Iteration: %d ; error = %.4f; " % (i+1, nrmse))
            if loss_hist[nr_validation - 1] - loss_hist[1] >= 0:
              print(i)
              print(32*"-")
              print(loss_hist)
              return (propr,i, loss_hist[nr_validation - 1])
              break

          if (i+1) % 25 == 0:
            print("Iteration: %d ; error = %.4f; " % (i+1, nrmse))


        return training_process

    def mse(self):
      predictions_matrix = tf.gather_nd(self.full_matrix(),self.R.indices)
      predictions_matrix = build_sparse_tensor(predictions_matrix, self.R.indices, self.R.dense_shape)
      mse = tf.keras.losses.MeanSquaredError()
      return mse(tf.sparse.to_dense(self.R), tf.sparse.to_dense(predictions_matrix)).numpy()

    def regularizationL1(self):
      abs_P = tf.abs(self.P)
      abs_Q = tf.abs(self.Q)
      sum_P = tf.reduce_sum(abs_P)
      sum_Q = tf.reduce_sum(abs_Q)

      return (sum_Q + sum_P) * self.beta

    def regularizationL2(self):
      pow_P = tf.pow(self.P,2)
      pow_Q = tf.pow(self.Q,2)
      sum_P = tf.reduce_sum(pow_P)
      sum_Q = tf.reduce_sum(pow_Q)

      return (sum_Q + sum_P) * self.beta

     # Full user-movie matrix
    def full_matrix(self):
        return tf.matmul(self.P,self.Q)


In [12]:
R= np.array(train_data.pivot(index = 'userId', columns ='movieId', values = 'rating').fillna(0))
indices = [[i, j] for i in range(R.shape[0]) for j in range(R.shape[1]) if R[i, j] > 0]
shape = R.shape
R = build_sparse_tensor(np.array(train_data.rating, dtype=np.float32), indices, shape)

In [13]:
# mf = MF_tensorflow(R, K=100, alpha=0.5, iterations=260, beta = 0.0001)
# training_process = mf.train()  
# print()
# print("P x Q:")
# print(mf.full_matrix())
# print()

Evaluation for float32 values:

In [14]:
# test_tensor = np.array(test_data.pivot(index = 'userId', columns ='movieId', values = 'rating').fillna(0))
# indices = [[i, j] for i in range(test_tensor.shape[0]) for j in range(test_tensor.shape[1]) if test_tensor[i, j] > 0]
# shape = test_tensor.shape
# test_tensor = build_sparse_tensor(np.array(test_data.rating, dtype=np.float32), indices, shape)
# test_tensor

In [15]:
# predictions_matrix = build_sparse_tensor(tf.gather_nd(mf.full_matrix(),test_tensor.indices), test_tensor.indices, test_tensor.dense_shape)
# m = tf.keras.metrics.RootMeanSquaredError()
# error = tf.reduce_sum(tf.losses.mean_squared_error(tf.sparse.to_dense(test_tensor), tf.sparse.to_dense(predictions_matrix)))
# res = m.update_state(tf.sparse.to_dense(test_tensor), tf.sparse.to_dense(predictions_matrix))
# print(m.result().numpy())
# error

In [16]:
# loss = tf.reduce_sum(tf.keras.losses.mean_absolute_percentage_error(tf.sparse.to_dense(test_tensor), tf.sparse.to_dense(predictions_matrix)))
# loss

In [17]:
# loss = tf.reduce_sum(tf.keras.losses.mean_absolute_error(tf.sparse.to_dense(test_tensor), tf.sparse.to_dense(predictions_matrix)))
# loss 

Evaluation for int values:

In [18]:
# round_test = build_sparse_tensor(np.array(test_data.rating, dtype=np.int), indices, shape)
# round_test.values

In [19]:
# predictions_matrix = build_sparse_tensor(tf.gather_nd(mf.full_matrix(),test_tensor.indices), test_tensor.indices, test_tensor.dense_shape)
# round_predictions_matrix = tf.round(tf.sparse.to_dense(predictions_matrix))
# round_predictions_matrix

In [20]:
# m = tf.keras.metrics.RootMeanSquaredError()
# error = tf.reduce_sum(tf.losses.mean_squared_error(tf.sparse.to_dense(round_test), round_predictions_matrix))
# res = m.update_state(tf.sparse.to_dense(test_tensor), round_predictions_matrix)
# print(m.result().numpy())
# error

In [21]:
# loss = tf.reduce_sum(tf.keras.losses.mean_absolute_percentage_error(tf.sparse.to_dense(test_tensor),round_predictions_matrix)
# loss

Cross validations:

In [ ]:
iters = 50000
interval_beta = [0.0001,0.0003,0.0005, 0.001,0.003, 0.005]
interval_alpha = [0.0003, 0.001,0.003, 0.005,0.025,0.05,0.03, 0.1,0.25,0.5,1,3,5,10]
interval_K = [20,50,60,80,100, 150] 
# visits = [(20, 0.00001, 0.0001), (50, 0.00001, 0.0001), (60, 0.00001, 0.0001), (80, 0.00001, 0.0001),(100, 0.00001, 0.0001),(150, 0.00001, 0.0001), (20, 0.00001, 0.001), (50, 0.00001, 0.001),(60, 0.00001, 0.001), (80, 0.00001, 0.001),(100, 0.00001, 0.001),(150, 0.00001, 0.001), (20, 0.00001, 0.005),(50, 0.00001, 0.005),(60, 0.00001, 0.005),(80, 0.00001, 0.005),(100, 0.00001, 0.005),(150, 0.00001, 0.005),(20, 0.00001, 0.1),(50, 0.00001, 0.1),(60, 0.00001, 0.1),(80, 0.00001, 0.1),(100, 0.00001, 0.1),(150, 1e-05, 0.1),(20, 1e-05, 0.5),(50, 1e-05, 0.5),(60, 1e-05, 0.5), (80, 1e-05, 0.5), (100, 1e-05, 0.5),(150, 1e-05, 0.5),(20, 1e-05, 1),(50, 1e-05, 1),(60, 1e-05, 1),(80, 1e-05, 1),(100, 1e-05, 1),(150, 1e-05, 1),(20, 1e-05, 5),(50, 1e-05, 5),(60, 1e-05, 5),(80, 1e-05, 5),(100, 1e-05, 5),(150, 1e-05, 5),(20, 1e-05, 10), (50, 1e-05, 10),(60, 1e-05, 10),(80, 1e-05, 10),(100, 1e-05, 10),(150, 1e-05, 10), (20, 0.0001, 0.0001), (50, 0.0001, 0.0001),(60, 0.0001, 0.0001),(80, 0.0001, 0.0001),(100, 0.0001, 0.0001),(150, 0.0001, 0.0001),(20, 0.0001, 0.001),(50, 0.0001, 0.001),(60, 0.0001, 0.001),(80, 0.0001, 0.001),(100, 0.0001, 0.001),(150, 0.0001, 0.001),(20, 0.0001, 0.005),(50, 0.0001, 0.005),(60, 0.0001, 0.005),(80, 0.0001, 0.005),(100, 0.0001, 0.005),(150, 0.0001, 0.005),(20, 0.0001, 0.1),(50, 0.0001, 0.1),(60, 0.0001, 0.1), (80, 0.0001, 0.1),(100, 0.0001, 0.1),(150, 0.0001, 0.1),(20, 0.0001, 0.5),(50, 0.0001, 0.5),(60, 0.0001, 0.5),(80, 0.0001, 0.5),(100, 0.0001, 0.5), (150, 0.0001, 0.5),(20, 0.0001, 1),(50, 0.0001, 1),(60, 0.0001, 1),(80, 0.0001, 1),(100, 0.0001, 1),(150, 0.0001, 1),(20, 0.0001, 5),(50, 0.0001, 5),(60, 0.0001, 5),(80, 0.0001, 5),(100, 0.0001, 5),(150, 0.0001, 5),(20, 0.0001, 10),(50, 0.0001, 10),(60, 0.0001, 10), (80, 0.0001, 10),(100, 0.0001, 10),(150, 0.0001, 10),(20, 0.001, 0.0001),(50, 0.001, 0.0001),(60, 0.001, 0.0001),(80, 0.001, 0.0001),(100, 0.001, 0.0001),(150, 0.001, 0.0001),(20, 0.001, 0.001),(50, 0.001, 0.001),(60, 0.001, 0.001), (80, 0.001, 0.001),(100, 0.001, 0.001), (150, 0.001, 0.001),(20, 0.001, 0.005),(50, 0.001, 0.005),(60, 0.001, 0.005),(80, 0.001, 0.005),(100, 0.001, 0.005),(150, 0.001, 0.005),(20, 0.001, 0.1),(50, 0.001, 0.1),(60, 0.001, 0.1),(80, 0.001, 0.1),(100, 0.001, 0.1),(150, 0.001, 0.1),(20, 0.001, 0.5),(50, 0.001, 0.5),(60, 0.001, 0.5),(80, 0.001, 0.5),(100, 0.001, 0.5),(150, 0.001, 0.5),(20, 0.001, 1),(50, 0.001, 1),(60, 0.001, 1),(80, 0.001, 1),(100, 0.001, 1),(150, 0.001, 1),(20, 0.001, 5),(50, 0.001, 5)]
m = 0

# visits = [(20, 0.0003, 0.0001),(50, 0.0003, 0.0001),(60, 0.0003, 0.0001),(80, 0.0003, 0.0001),(100, 0.0003, 0.0001),(150, 0.0003, 0.0001),(20, 0.0003, 0.0003),(50, 0.0003, 0.0003),(60, 0.0003, 0.0003),(80, 0.0003, 0.0003),(100, 0.0003, 0.0003),(150, 0.0003, 0.0003),(20, 0.0003, 0.0005),(50, 0.0003, 0.0005),(60, 0.0003, 0.0005),(80, 0.0003, 0.0005),(100, 0.0003, 0.0005),(150, 0.0003, 0.0005),(20, 0.0003, 0.001),(50, 0.0003, 0.001)]

founded_var = []

for i in interval_alpha:
  for j in interval_beta:
    for k in interval_K:
#       if( m < len(visits) and i == visits[m][1] and j  == visits[m][2] and k == visits[m][0]):
#         m+=1
#         print("in range do nothing")
#       else:
        print("learning rate: %.5f, regularisation: %.4f, latent feautures: %d" % (i , j , k))
        mf = MF_tensorflow(R, K=k, alpha=i, iterations=iters, beta = j)
        var = mf.train()
        x = mf.full_matrix()
        
        lower_tensor = tf.greater(x, 0.)
        upper_tensor = tf.less(x, 5.)
        in_range = tf.logical_and(lower_tensor, upper_tensor)
        procent = tf.reduce_sum(tf.cast(in_range, tf.float32))/ (x.shape[0] * x.shape[1])
        
        test_array = np.array(test_data.pivot(index = 'userId', columns ='movieId', values = 'rating').fillna(0))
        
        indices = [[f, g] for f in range(test_array.shape[0]) for g in range(test_array.shape[1]) if test_array[f, g] > 0]
        
        test_tensor = build_sparse_tensor(np.array(test_data.rating, dtype=np.float32), indices, shape)

        round_test = build_sparse_tensor(np.array(test_data.rating, dtype=np.int), indices, shape)
        predictions_matrix = build_sparse_tensor(tf.gather_nd(mf.full_matrix(),test_tensor.indices), test_tensor.indices, test_tensor.dense_shape)
        round_predictions_matrix = tf.round(tf.sparse.to_dense(predictions_matrix))
        error = tf.reduce_sum(tf.losses.mean_squared_error(tf.sparse.to_dense(round_test), round_predictions_matrix))
        
        founded_var.append((var, procent.numpy() * 100, error))
        with open('hyperparametersL2.txt', 'w') as writefile:
          for line in founded_var:
              print(line)
              writefile.write(str(line)+"\n")
        print(32*"*")  



learning rate: 0.00030, regularisation: 0.0001, latent feautures: 20
Iteration: 25 ; error = 38.2917; 
Iteration: 50 ; error = 37.6982; 
Iteration: 75 ; error = 37.1225; 
Iteration: 100 ; error = 36.5641; 
Iteration: 125 ; error = 36.0223; 
Iteration: 150 ; error = 35.4962; 
Iteration: 175 ; error = 34.9853; 
Iteration: 200 ; error = 34.4888; 
Iteration: 225 ; error = 34.0062; 
Iteration: 250 ; error = 33.5367; 
Iteration: 275 ; error = 33.0799; 
Iteration: 300 ; error = 32.6352; 
Iteration: 325 ; error = 32.2021; 
Iteration: 350 ; error = 31.7800; 
Iteration: 375 ; error = 31.3685; 
Iteration: 400 ; error = 30.9672; 
Iteration: 425 ; error = 30.5755; 
Iteration: 450 ; error = 30.1930; 
Iteration: 475 ; error = 29.8193; 
Iteration: 500 ; error = 29.4539; 
Iteration: 525 ; error = 29.0965; 
Iteration: 550 ; error = 28.7467; 
Iteration: 575 ; error = 28.4040; 
Iteration: 600 ; error = 28.0680; 
Iteration: 625 ; error = 27.7384; 
Iteration: 650 ; error = 27.4147; 
Iteration: 675 ; error =

Iteration: 5800 ; error = 2.4282; 
Iteration: 5825 ; error = 2.4151; 
Iteration: 5850 ; error = 2.4022; 
Iteration: 5875 ; error = 2.3895; 
Iteration: 5900 ; error = 2.3769; 
Iteration: 5925 ; error = 2.3645; 
Iteration: 5950 ; error = 2.3523; 
Iteration: 5975 ; error = 2.3402; 
Iteration: 6000 ; error = 2.3283; 
Iteration: 6025 ; error = 2.3165; 
Iteration: 6050 ; error = 2.3049; 
Iteration: 6075 ; error = 2.2935; 
Iteration: 6100 ; error = 2.2822; 
Iteration: 6125 ; error = 2.2710; 
Iteration: 6150 ; error = 2.2600; 
Iteration: 6175 ; error = 2.2491; 
Iteration: 6200 ; error = 2.2384; 
Iteration: 6225 ; error = 2.2278; 
Iteration: 6250 ; error = 2.2174; 
Iteration: 6275 ; error = 2.2071; 
Iteration: 6300 ; error = 2.1969; 
Iteration: 6325 ; error = 2.1868; 
Iteration: 6350 ; error = 2.1769; 
Iteration: 6375 ; error = 2.1671; 
Iteration: 6400 ; error = 2.1574; 
Iteration: 6425 ; error = 2.1479; 
Iteration: 6450 ; error = 2.1384; 
Iteration: 6475 ; error = 2.1291; 
Iteration: 6500 ; er

Iteration: 11625 ; error = 1.3973; 
Iteration: 11650 ; error = 1.3966; 
Iteration: 11675 ; error = 1.3958; 
Iteration: 11700 ; error = 1.3951; 
Iteration: 11725 ; error = 1.3943; 
Iteration: 11750 ; error = 1.3936; 
Iteration: 11775 ; error = 1.3929; 
Iteration: 11800 ; error = 1.3922; 
Iteration: 11825 ; error = 1.3915; 
Iteration: 11850 ; error = 1.3908; 
Iteration: 11875 ; error = 1.3902; 
Iteration: 11900 ; error = 1.3895; 
Iteration: 11925 ; error = 1.3889; 
Iteration: 11950 ; error = 1.3882; 
Iteration: 11975 ; error = 1.3876; 
Iteration: 12000 ; error = 1.3870; 
Iteration: 12025 ; error = 1.3864; 
Iteration: 12050 ; error = 1.3858; 
Iteration: 12075 ; error = 1.3852; 
Iteration: 12100 ; error = 1.3846; 
Iteration: 12125 ; error = 1.3841; 
Iteration: 12150 ; error = 1.3835; 
Iteration: 12175 ; error = 1.3830; 
Iteration: 12200 ; error = 1.3824; 
Iteration: 12225 ; error = 1.3819; 
Iteration: 12250 ; error = 1.3814; 
Iteration: 12275 ; error = 1.3809; 
Iteration: 12300 ; error = 1

Iteration: 17325 ; error = 1.3518; 
Iteration: 17350 ; error = 1.3518; 
Iteration: 17375 ; error = 1.3518; 
Iteration: 17400 ; error = 1.3518; 
Iteration: 17425 ; error = 1.3518; 
Iteration: 17450 ; error = 1.3518; 
Iteration: 17475 ; error = 1.3518; 
Iteration: 17500 ; error = 1.3517; 
Iteration: 17525 ; error = 1.3517; 
Iteration: 17550 ; error = 1.3517; 
Iteration: 17575 ; error = 1.3517; 
Iteration: 17600 ; error = 1.3517; 
Iteration: 17625 ; error = 1.3517; 
Iteration: 17650 ; error = 1.3517; 
Iteration: 17675 ; error = 1.3517; 
Iteration: 17700 ; error = 1.3517; 
Iteration: 17725 ; error = 1.3517; 
Iteration: 17750 ; error = 1.3516; 
Iteration: 17775 ; error = 1.3516; 
Iteration: 17800 ; error = 1.3516; 
Iteration: 17825 ; error = 1.3516; 
Iteration: 17850 ; error = 1.3516; 
Iteration: 17875 ; error = 1.3516; 
Iteration: 17900 ; error = 1.3516; 
Iteration: 17925 ; error = 1.3516; 
Iteration: 17950 ; error = 1.3516; 
Iteration: 17975 ; error = 1.3516; 
Iteration: 18000 ; error = 1

Iteration: 2900 ; error = 10.6418; 
Iteration: 2925 ; error = 10.5096; 
Iteration: 2950 ; error = 10.3798; 
Iteration: 2975 ; error = 10.2525; 
Iteration: 3000 ; error = 10.1275; 
Iteration: 3025 ; error = 10.0048; 
Iteration: 3050 ; error = 9.8844; 
Iteration: 3075 ; error = 9.7661; 
Iteration: 3100 ; error = 9.6499; 
Iteration: 3125 ; error = 9.5358; 
Iteration: 3150 ; error = 9.4237; 
Iteration: 3175 ; error = 9.3135; 
Iteration: 3200 ; error = 9.2053; 
Iteration: 3225 ; error = 9.0990; 
Iteration: 3250 ; error = 8.9944; 
Iteration: 3275 ; error = 8.8917; 
Iteration: 3300 ; error = 8.7906; 
Iteration: 3325 ; error = 8.6913; 
Iteration: 3350 ; error = 8.5936; 
Iteration: 3375 ; error = 8.4976; 
Iteration: 3400 ; error = 8.4031; 
Iteration: 3425 ; error = 8.3101; 
Iteration: 3450 ; error = 8.2187; 
Iteration: 3475 ; error = 8.1288; 
Iteration: 3500 ; error = 8.0402; 
Iteration: 3525 ; error = 7.9531; 
Iteration: 3550 ; error = 7.8674; 
Iteration: 3575 ; error = 7.7830; 
Iteration: 360

...

Iteration: 25 ; error = 856.2443; 
Iteration: 50 ; error = 819.0582; 
Iteration: 75 ; error = 784.0559; 
Iteration: 100 ; error = 751.0590; 
Iteration: 125 ; error = 719.8877; 
Iteration: 150 ; error = 690.3858; 
Iteration: 175 ; error = 662.4181; 
Iteration: 200 ; error = 635.8667; 
Iteration: 225 ; error = 610.6276; 
Iteration: 250 ; error = 586.6087; 
Iteration: 275 ; error = 563.7274; 
Iteration: 300 ; error = 541.9094; 
Iteration: 325 ; error = 521.0879; 
Iteration: 350 ; error = 501.2018; 
Iteration: 375 ; error = 482.1957; 
Iteration: 400 ; error = 464.0191; 
Iteration: 425 ; error = 446.6251; 
Iteration: 450 ; error = 429.9710; 
Iteration: 475 ; error = 414.0172; 
Iteration: 500 ; error = 398.7268; 
Iteration: 525 ; error = 384.0661; 
Iteration: 550 ; error = 370.0032; 
Iteration: 575 ; error = 356.5087; 
Iteration: 600 ; error = 343.5549; 
Iteration: 625 ; error = 331.1162; 
Iteration: 650 ; error = 319.1682; 
Iteration: 675 ; error = 307.6884; 
Iteration: 700 ; error = 296.65

Iteration: 5725 ; error = 11.0980; 
Iteration: 5750 ; error = 11.0970; 
Iteration: 5775 ; error = 11.0959; 
Iteration: 5800 ; error = 11.0950; 
Iteration: 5825 ; error = 11.0941; 
Iteration: 5850 ; error = 11.0932; 
Iteration: 5875 ; error = 11.0924; 
Iteration: 5900 ; error = 11.0917; 
Iteration: 5925 ; error = 11.0910; 
Iteration: 5950 ; error = 11.0904; 
Iteration: 5975 ; error = 11.0898; 
Iteration: 6000 ; error = 11.0892; 
Iteration: 6025 ; error = 11.0887; 
Iteration: 6050 ; error = 11.0882; 
Iteration: 6075 ; error = 11.0877; 
Iteration: 6100 ; error = 11.0873; 
Iteration: 6125 ; error = 11.0869; 
Iteration: 6150 ; error = 11.0866; 
Iteration: 6175 ; error = 11.0862; 
Iteration: 6200 ; error = 11.0859; 
Iteration: 6225 ; error = 11.0856; 
Iteration: 6250 ; error = 11.0853; 
Iteration: 6275 ; error = 11.0851; 
Iteration: 6300 ; error = 11.0848; 
Iteration: 6325 ; error = 11.0846; 
Iteration: 6350 ; error = 11.0844; 
Iteration: 6375 ; error = 11.0842; 
Iteration: 6400 ; error = 11

Iteration: 25 ; error = 1271.2386; 
Iteration: 50 ; error = 1215.2964; 
Iteration: 75 ; error = 1162.7610; 
Iteration: 100 ; error = 1113.3281; 
Iteration: 125 ; error = 1066.6995; 
Iteration: 150 ; error = 1022.6197; 
Iteration: 175 ; error = 980.8710; 
Iteration: 200 ; error = 941.2656; 
Iteration: 225 ; error = 903.6390; 
Iteration: 250 ; error = 867.8469; 
Iteration: 275 ; error = 833.7609; 
Iteration: 300 ; error = 801.2665; 
Iteration: 325 ; error = 770.2611; 
Iteration: 350 ; error = 740.6516; 
Iteration: 375 ; error = 712.3539; 
Iteration: 400 ; error = 685.2914; 
Iteration: 425 ; error = 659.3937; 
Iteration: 450 ; error = 634.5966; 
Iteration: 475 ; error = 610.8408; 
Iteration: 500 ; error = 588.0717; 
Iteration: 525 ; error = 566.2382; 
Iteration: 550 ; error = 545.2936; 
Iteration: 575 ; error = 525.1940; 
Iteration: 600 ; error = 505.8984; 
Iteration: 625 ; error = 487.3688; 
Iteration: 650 ; error = 469.5692; 
Iteration: 675 ; error = 452.4662; 
Iteration: 700 ; error = 

Iteration: 5700 ; error = 11.1075; 
Iteration: 5725 ; error = 11.1058; 
Iteration: 5750 ; error = 11.1042; 
Iteration: 5775 ; error = 11.1027; 
Iteration: 5800 ; error = 11.1013; 
Iteration: 5825 ; error = 11.1000; 
Iteration: 5850 ; error = 11.0988; 
Iteration: 5875 ; error = 11.0976; 
Iteration: 5900 ; error = 11.0965; 
Iteration: 5925 ; error = 11.0955; 
Iteration: 5950 ; error = 11.0946; 
Iteration: 5975 ; error = 11.0937; 
Iteration: 6000 ; error = 11.0929; 
Iteration: 6025 ; error = 11.0921; 
Iteration: 6050 ; error = 11.0914; 
Iteration: 6075 ; error = 11.0907; 
Iteration: 6100 ; error = 11.0901; 
Iteration: 6125 ; error = 11.0895; 
Iteration: 6150 ; error = 11.0889; 
Iteration: 6175 ; error = 11.0884; 
Iteration: 6200 ; error = 11.0880; 
Iteration: 6225 ; error = 11.0875; 
Iteration: 6250 ; error = 11.0871; 
Iteration: 6275 ; error = 11.0867; 
Iteration: 6300 ; error = 11.0864; 
Iteration: 6325 ; error = 11.0860; 
Iteration: 6350 ; error = 11.0857; 
Iteration: 6375 ; error = 11

Iteration: 25 ; error = 280.2419; 
Iteration: 50 ; error = 269.0573; 
Iteration: 75 ; error = 258.4390; 
Iteration: 100 ; error = 248.3531; 
Iteration: 125 ; error = 238.7619; 
Iteration: 150 ; error = 229.6315; 
Iteration: 175 ; error = 220.9318; 
Iteration: 200 ; error = 212.6354; 
Iteration: 225 ; error = 204.7177; 
Iteration: 250 ; error = 197.1563; 
Iteration: 275 ; error = 189.9307; 
Iteration: 300 ; error = 183.0219; 
Iteration: 325 ; error = 176.4125; 
Iteration: 350 ; error = 170.0866; 
Iteration: 375 ; error = 164.0292; 
Iteration: 400 ; error = 158.2264; 
Iteration: 425 ; error = 152.6653; 
Iteration: 450 ; error = 147.3340; 
Iteration: 475 ; error = 142.2211; 
Iteration: 500 ; error = 137.3161; 
Iteration: 525 ; error = 132.6091; 
Iteration: 550 ; error = 128.0909; 
Iteration: 575 ; error = 123.7526; 
Iteration: 600 ; error = 119.5862; 


In [ ]:
founded_var

In [ ]:
# x = mf.full_matrix()
# # x = [[1,2,3],[7,8,9],[-1,-3,-4]]
# lower_tensor = tf.greater(x, 0.)
# upper_tensor = tf.less(x, 5.)
# in_range = tf.logical_and(lower_tensor, upper_tensor)
# count = tf.reduce_sum(tf.cast(in_range, tf.float32))/ (x.shape[0] * x.shape[1])

In [ ]:
# procent = count / (x.shape[0] * x.shape[1])
# procent.numpy() * 100

Read info about movies.

In [ ]:
movies_data  =  pd.read_csv('./content/u.item', encoding= 'latin-1', sep = '|', 
  names = ['movieId','movieName','releaseDate','nothing','imdbLink','unkownGenre','actionGenre','adventureGenre','animationGenre','childrensGenre','comedyGenre','crimeGenre','documentaryGenre','dramaGenre','fantasyGenre','filmNoirGenre','horrorGenre','musicalGenre','mysteryGenre','romanceGenre','sciFiGenre','thrillerGenre','warGenre','westernGenre'])
movies_data.head()

In [ ]:
movies_genres = []

for index, row in movies_data.iterrows():
  genres_string = ''
  if row['unkownGenre'] == 1:
    genres_string+='unkown '
  if row['actionGenre'] == 1:
    genres_string+='action '
  if row['adventureGenre'] == 1:
    genres_string+='adventure '
  if row['animationGenre'] == 1:
    genres_string+='animation '
  if row['childrensGenre'] == 1:
    genres_string+='childrens '
  if row['comedyGenre'] == 1:
    genres_string+='comedy '
  if row['crimeGenre'] == 1:
    genres_string+='crime '
  if row['documentaryGenre'] == 1:
    genres_string+='documentary '
  if row['dramaGenre'] == 1:
    genres_string+='drama '
  if row['fantasyGenre'] == 1:
    genres_string+='fantasy '
  if row['filmNoirGenre'] == 1:
    genres_string+='filmNoir '
  if row['horrorGenre'] == 1:
    genres_string+='horror '
  if row['musicalGenre'] == 1:
    genres_string+='musical '
  if row['mysteryGenre'] == 1:
    genres_string+='mystery '
  if row['romanceGenre'] == 1:
    genres_string+='romance '
  if row['sciFiGenre'] == 1:
    genres_string+='sciFi '
  if row['thrillerGenre'] == 1:
    genres_string+='thriller '
  if row['warGenre'] == 1:
    genres_string+='war '
  if row['westernGenre'] == 1:
    genres_string+='western '
  
  movies_genres.append(genres_string)
  genres_string = ''

In [ ]:
movies_data['genres'] = movies_genres
del movies_data['unkownGenre']
del movies_data['actionGenre']
del movies_data['adventureGenre']
del movies_data['westernGenre']
del movies_data['warGenre']
del movies_data['thrillerGenre']
del movies_data['sciFiGenre']
del movies_data['romanceGenre']
del movies_data['mysteryGenre']
del movies_data['musicalGenre']
del movies_data['horrorGenre']
del movies_data['filmNoirGenre']
del movies_data['fantasyGenre']
del movies_data['dramaGenre']
del movies_data['documentaryGenre']
del movies_data['crimeGenre']
del movies_data['comedyGenre']
del movies_data['childrensGenre']
del movies_data['animationGenre']
del movies_data['nothing']

movies_data.head()